In [69]:
import pandas as pd
from bs4 import BeautifulSoup as soup
import requests
from selenium import webdriver
import time
from random import random

In [70]:
# Scraping "free-proxy-list.net" to get a list of proxies.
res = requests.get("https://free-proxy-list.net")
proxies = pd.read_html(res.text)
proxies = proxies[0][:-1]
proxies.columns = [c.replace(' ', '_') for c in proxies.columns]
ip = list(proxies.IP_Address)
port = list(map(str, list(map(int, list(proxies.Port)))))
print(ip)
print(port)

['200.116.198.177', '200.52.141.162', '138.121.32.26', '178.213.130.159', '91.206.210.54', '79.129.117.118', '187.111.160.6', '94.232.11.178', '170.82.231.26', '89.42.133.58', '109.232.106.236', '118.175.93.189', '96.255.50.185', '190.0.53.30', '158.255.23.146', '118.200.73.91', '103.58.74.2', '45.235.86.1', '196.53.25.87', '110.168.26.171', '190.210.8.91', '196.54.30.246', '59.153.84.254', '103.224.185.20', '196.54.30.61', '196.54.30.56', '138.122.140.35', '196.54.30.14', '176.226.251.131', '102.164.214.225', '117.58.241.164', '196.53.25.49', '196.53.25.14', '196.53.25.203', '64.17.30.238', '157.119.207.13', '196.53.25.102', '95.84.128.25', '88.198.33.232', '188.133.165.206', '196.54.30.73', '196.54.30.147', '213.6.101.174', '49.0.1.44', '195.189.71.187', '182.52.51.10', '37.152.189.111', '178.215.190.240', '196.54.30.239', '196.54.30.69', '196.54.30.160', '186.159.2.241', '196.54.30.253', '196.54.30.107', '196.53.25.8', '46.98.36.250', '78.188.65.105', '80.87.184.49', '117.103.5.186'

In [71]:
def conf_webdriver(proxy_id):    # Configuring the Proxy in Firefox webdriver.
    PROXY = ip[proxy_id] + ":" + port[proxy_id]
    webdriver.DesiredCapabilities.FIREFOX['proxy']={
        "sslProxy":PROXY,
        "ftpProxy":PROXY,
        "socksProxy":PROXY,
        "socksVersion":5,
        "proxyType":"MANUAL"
    }
    print("Change proxy: " + PROXY)

# Preparing the variables that will contain the data.
category_data = []
localite_data = []
criterion_data = []
price_data = []
monthly_data = []
description_data = []
plus_data = []
energie_data = []

conf_webdriver(0)

max = 20    # The maximum seconds of delay.
min = 11    # The minimum seconds of delay.

page_num1=60  # Current page number.
page_num2=page_num1+1   # Modify the number of pages to scrape.

# Loop the pages.
proxy_id = 1
for num in range(page_num1,page_num2):   
    driver = webdriver.Firefox()    # Open Firefox webdriver.
    
    # ----------- Testing the availability of a Proxy. -------------
    cond = 0
    while(cond == 0):   
        try:
            driver.get("https://seloger.com/list.htm?projects=2%2C5&types=1%2C2%2C4%2C3%2C9%2C13%2C14%2C12%2C11%2C10&natures=1%2C2%2C4&places=%5B%7BidPays%3A250%7D%5D&enterprise=0&houseboat=1&qsVersion=1.0&LISTING-LISTpg="+str(num))
            cond = 1
        except:
            driver.quit()
            conf_webdriver(proxy_id)
            driver = webdriver.Firefox()
            proxy_id = proxy_id+1
    # -------------------------------------------------------------        
    
    # ----- Adding a random delay. -----
    time_random1 = random()
    time1 = min + (time_random1 * (max - min))
    time.sleep(time1)
    # ---------------------------
    
    url2 = str(driver.current_url)   # Get the current url.
    
    # ------ When seloger.com find out that you are a bot, we will change the page and the Proxy. ------
    if('validate.perfdrive.com' in url2):  
        driver.quit()
        proxy_id +=1 
        conf_webdriver(proxy_id)
        driver = webdriver.Firefox()
        continue
    # ----------------------------------------------------------------------------------------------
    
    page = soup(driver.page_source, 'html.parser')  # Get the HTML code of the current page.
    house_containers = page.find_all("div", class_="block__ShadowedBlock-sc-10w6hsj-0 ListContent__SmartClassifiedExtended-sc-1viyr2k-2 iddbNe classified__ClassifiedContainer-sc-1wmlctl-0 cGCTai Card__CardContainer-sc-7insep-7 kjDrzL")
    print("Number of ads: " + str(len(house_containers)))  # The number of ads in a single page (25).
    
    driver2 = webdriver.Firefox()   # Open a second Firefox webdriver.
    proxy_id2 = 1
    
    # Loop the ads.
    for data in house_containers:
        find_link = data.find_all("a", class_="CoveringLink-a3s3kt-0 dXJclF")    # Getting the ad link.
        L=str(find_link[0]).split(" ")
        ad_link=L[3][14:-1]
               
        if ('seloger.com' not in ad_link):  # If the ad isn't in seloger.com, we will skip it.
            continue
        
        # ----------- Testing the availability of a new Proxy. -------------
        cond2 = 0
        while(cond2 == 0):  
            try:
                driver2.get("https://" + ad_link)
                cond2 = 1
            except:
                driver2.quit()
                conf_webdriver(proxy_id2)
                driver2 = webdriver.Firefox()
                proxy_id2 = proxy_id2+1
        # -----------------------------------------------------------------
        
        # ----- Adding a random delay. -----
        time_random2 = random()
        time2 = min + (time_random2 * (max - min))
        time.sleep(time2)
        # ---------------------------
        
        url = str(driver2.current_url)     # Get the current url.
       
        # ------ When seloger.com find out that you are a bot, we will change the ad and the Proxy. ------
        if('validate.perfdrive.com' in url): 
            driver2.quit()
            proxy_id2 +=1 
            conf_webdriver(proxy_id2)
            driver2 = webdriver.Firefox()     
            continue
        # -----------------------------------------------------------------------------------------------
            
        page2 = soup(driver2.page_source, 'html.parser')  # Get the HTML code of the ad.
        
        criterion = []
        plus = []
        energie =[]
        
        # Getting the category.
        try:   
            try:    
                try:    # The first version of seloger.com.
                    find_category = page2.find_all("h2", class_="c-h2")[0]
                    category = str(find_category)[len('<h2 class="c-h2">'):-5]
                except IndexError:   # The second version of seloger.com.
                    find_category = page2.find_all("div", class_="Summary__Text-sc-1wkzvu-6 gcWjRm")[0]
                    category=(str(find_category)[len('<div class="Summary__Text-sc-1wkzvu-6 gcWjRm">'):-6])
            except:   # The third version of seloger.com.
                find_category = page2.find_all("div", class_="Summarystyled__Title-tzuaot-3 dkepid")[0]
                category=(str(find_category)[len('<div class="Summarystyled__Title-tzuaot-3 dkepid">'):-6])
        except:   # When the category doesn't exist.
            category = "NULL"
            
        # Getting the City.
        try:
            try:
                try:
                    find_localite = page2.find_all("p", class_="localite")[0]
                    localite = str(find_localite)[len('<p class="localite">'):-4]
                except IndexError:
                    find_localite = page2.find_all("div", class_="Summary__Text-sc-1wkzvu-6 gcWjRm")[1]
                    localite=(str(find_localite)[len('<div class="Summary__Text-sc-1wkzvu-6 gcWjRm">'):-6])
            except:
                find_localite = page2.find_all("div", class_="Summarystyled__Address-tzuaot-5 fLUFm")[0]
                localite=(str(find_localite)[len('<div class="Summarystyled__Address-tzuaot-5 fLUFm">'):-6])
        except:
            localite = "NULL"
            
        # Getting the criterion (There are only two versions of seloger.com for the criterion).
        try:
            try:
                find_criterion = page2.find_all("ul", class_="criterion")[0]
                criterion2 = str(find_criterion).split('<li>')[1:]
                for i in range (len(criterion2)):
                    ch=""
                    for j in range (len(criterion2[i])):
                        if(str(criterion2[i])[j]=="<"):
                            break
                        ch=ch+str(criterion2[i])[j]
                    criterion.append(ch)

            except IndexError:
                find_criterion = page2.find_all("div", class_="Tags__TagContainer-sc-1gccxml-0 kQEoGa")[:3]
                find_criterion = list(map(str, find_criterion))
                for i in range (len(find_criterion)):
                    criterion.append(find_criterion[i][len('<div class="Tags__TagContainer-sc-1gccxml-0 kQEoGa">'):-6])

        except:
            criterion.append("NULL")
            
        # Getting the price.
        try:
            try:
                try:
                    find_price = page2.find_all("a", class_="js-smooth-scroll-link price")[0]
                    price = str(find_price)[len('<a class="js-smooth-scroll-link price" data-smoothscroll-offset="160" href="#about_price_anchor">'):-4].replace(" ","").replace("\n","")
                except IndexError:
                    find_price = page2.find_all("span", class_="global-styles__TextNoWrap-sc-1aeotog-7 iAJdaY")[0]
                    price = str(find_price)[len('<span class="global-styles__TextNoWrap-sc-1aeotog-7 iAJdaY">'):-7]
            except:
                find_price = page2.find_all("span", class_="global-styles__TextNoWrap-sc-1aeotog-6 dVzJN")[0]
                price = str(find_price)[len('<span class="global-styles__TextNoWrap-sc-1aeotog-6 dVzJN">'):-7]
        except:
            price = "NULL"
            
        # Getting the monthly price.
        try:
            try:
                try:
                    find_monthly = page2.find_all("a", class_="monthly")[0]
                    monthly = str(find_monthly)[len('<a class="monthly" href="https://e-immobilier.credit-agricole.fr/simulca/?ORI=selogercampagne&amp;xtor=AL-2-[partenariat]-[Seloger]-[pdd]-[lien_a_partir_sticky]" rel="nofollow" target="_blank">'):-len('<sup>1</sup></a>')]
                except IndexError:
                    find_monthly = page2.find_all("span", class_="global-styles__TextNoWrap-sc-1aeotog-7 iAJdaY")[1]
                    monthly = str(find_monthly)[len('<span class="global-styles__TextNoWrap-sc-1aeotog-7 iAJdaY">'):-7]
            except:
                find_monthly = page2.find_all("span", class_="global-styles__TextNoWrap-sc-1aeotog-6 dVzJN")[1]
                monthly = str(find_monthly)[len('<span class="global-styles__TextNoWrap-sc-1aeotog-6 dVzJN">'):-7]
        except:
            monthly = "NULL"

        # Getting the description.    
        try:
            try:
                find_description = page2.find_all("p", class_="sh-text-light")[0]
                description = str(find_description)[len('<p class="sh-text-light" id="js-descriptifBien">'):-4]
            except IndexError:
                find_description = page2.find_all("div", class_="ShowMoreText__UITextContainer-sc-5ggbbc-0 hCeOyd")[0]
                description = str(find_description)[len('<div class="ShowMoreText__UITextContainer-sc-5ggbbc-0 hCeOyd">'):-len('<button class="ContactLink__StyledButton-sc-1ex39dt-0 jPagdq">Demander plus d\'informations à l\'agence<div class="ContactLink__Chevron-sc-1ex39dt-1 gYbUwK Icon__IconElement-sc-1xnn3zd-0 fFMQXr" color="#e00034" data-test="contact-link-chevron" style="margin-left:5px"><svg viewbox="0 0 512 512"><path d="M144.9 495.9c-9 0-16.4-7.3-16.4-16.4 0-4.2 1.6-8.3 4.6-11.4L338 256 133.1 43.8c-6.2-6.5-6-16.9.5-23.2 6.5-6.2 16.8-6 23 .4l226.9 235-226.8 234.9c-3.1 3.2-7.4 5-11.8 5z"></path></svg></div></button>')]
        except:
            description = "NULL"
        
        # Getting the "more information".
        try:
            try:
                find_plus = page2.find_all("div", class_="u-left")[0]
                find_plus = page2.find_all("div", class_="u-left")
                for i in range (len(find_plus)):
                    plus2 = str(find_plus[i]).replace('<div class="u-left">', "").replace("</div>", "")
                    plus.append(plus2)

            except IndexError:
                find_plus = page2.find_all("figcaption") + page2.find_all("li", class_="GeneralList__Item-sc-9gtpjm-1 dsevYN")
                for i in range (len(find_plus)):
                    ch=""
                    for j in range (len(str(find_plus[i]))):
                        if(str(find_plus[i])[j]=='>'):
                            k=j
                            while(str(find_plus[i])[k]!="<"):
                                ch=ch + str(find_plus[i])[k]
                                k=k+1
                            break
                    plus.append(ch[1:])
        except:
            plus.append("NULL")

        # Getting the energy information
        try:
            try:
                find_energie = page2.find_all("div", class_="info-detail")[0]
                find_energie = page2.find_all("div", class_="info-detail")

                energie.append(str(find_energie[0])[len('<div class="info-detail">'):-6])
                energie.append(str(find_energie[1])[len('<div class="info-detail">'):-6])

            except IndexError:
                find_energie = page2.find_all("span", class_="Preview__PreviewTooltipValue-sc-1pa12ii-4 eGDcBO")
                energie.append(str(find_energie[0])[len('<span class="Preview__PreviewTooltipValue-sc-1pa12ii-4 eGDcBO">'):-7])
                energie.append(str(find_energie[1])[len('<span class="Preview__PreviewTooltipValue-sc-1pa12ii-4 eGDcBO">'):-7])
        except:
            energie.append("NULL")
            energie.append("NULL")


        print("Category: " + str(category))
        print("City: " + str(localite))
        print("Criterion: " + str(criterion))
        print("Price: " + str(price))
        print("Monthly: " + str(monthly))
        print("Description: " + str(description))
        print("More information: " + str(plus))
        print("Energy: " + str(energie))
        print("Page Number: " + str(num))
        print("-----------------------------------------------------")

        category_data.append(str(category))
        localite_data.append(str(localite))
        criterion_data.append(criterion)
        price_data.append(str(price))
        monthly_data.append(str(monthly))
        description_data.append(str(description))
        plus_data.append(plus)
        energie_data.append(energie)
    
    # Cleaning the criterion field
    for i in range (len(criterion_data)):
        if (len(criterion_data[i])==1):
            if("pièce" in criterion_data[i][0]):
                criterion_data[i].insert(1, "NULL")
                criterion_data[i].insert(2, "NULL")
            if("chambre" in criterion_data[i][0]):
                criterion_data[i].insert(0, "NULL")
                criterion_data[i].insert(2, "NULL")
            if("m²" in criterion_data[i][0]):
                criterion_data[i].insert(0, "NULL")
                criterion_data[i].insert(1, "NULL")
        if (len(criterion_data[i])==2):
            if(("pièce" in criterion_data[i][0]) and ("chambre" in criterion_data[i][1])):
                criterion_data[i].insert(2, "NULL")
            if(("pièce" in criterion_data[i][0]) and ("m²" in criterion_data[i][1])):
                criterion_data[i].insert(1, "NULL")
            if(("chambre" in criterion_data[i][0]) and ("m²" in criterion_data[i][1])):
                criterion_data[i].insert(0, "NULL")
        if(len(criterion_data[i])==0):
            criterion_data[i].append("NULL")
            criterion_data[i].append("NULL")
            criterion_data[i].append("NULL")
    
    # Saving the data.
    df = pd.DataFrame()
    df["category"] = category_data
    df["localite"] = localite_data
    df["piece"] = [i[0] for i in criterion_data]
    df["chambres"] = [i[1] for i in criterion_data]
    df["area"] = [i[2] for i in criterion_data]
    df["price"] = price_data
    df["monthly"] = monthly_data
    df["description"] = description_data
    df["Diagnostic de performance énergétique(kWhEP/m².an)"] = [i[0] for i in energie_data]
    df["Indice d'émission de gaz à effet de serre(kgeqCO2/m².an)"] = [i[1] for i in energie_data]
    df["plus"] = plus_data
    df["type"] = "Acheter"
    
    df.to_csv("./page.csv", index=False)
    csv1 = pd.read_csv("./page.csv")
    csv2 = pd.read_csv("./full_data.csv")
    full_data = pd.concat([csv1, csv2])
    full_data.to_csv("./full_data.csv", index=False)
    print('**DONE**')


Change proxy: 200.116.198.177:49951
Change proxy: 200.52.141.162:53281
Change proxy: 138.121.32.26:23500
Number of ads: 25
Category: Maison / Villa
City: Quartier <!-- -->Bassin Orsetti-Bassin du Midi<!-- -->, <!-- -->Sète
Criterion: ['6 pièces', '160 m²', '6 pièces']
Price: 928 000 €
Monthly: 3 833 €
</p><p> Vue imprenable sur la mer !</p><butto ou location de vacances, accès direct à la mer.uzzi, S/ 650 M² de jardin clos et arboré.
More information: ['Orientation Sud-Est', '3 Terrasses', 'Vue', 'Piscine', 'Surface de 160m²', 'Bâtiment de 1 étage', '8 Pièces', '6 Chambres', 'Année de construction 2007', 'Adapté PMR', '1 Salle de bain', "3 Salles d'eau", '3 Toilettes', 'Cuisine américaine équipée', 'Salle de séjour : 35m²', '3 Parkings', 'Orientation Est']
Energy: ['145', '7']
Page Number: 60
-----------------------------------------------------
Category: Maison / Villa
City: Quartier <!-- -->Centre Ville<!-- -->, <!-- -->Aigues-Mortes
Criterion: ['4 pièces', '115 m²', '4 pièces']
Pric

Category: Maison / Villa
City: Caveirac
Criterion: ['5 pièces', '186 m²', '6 pièces']
Price: 311 000 €
Monthly: 1 291 €
</p><p> MAGNIFIQUE OPPORTUNITÉ A L'ABRI DES REGARDS ! PRIX NÉGOCIABLE.. Corinne Hecquet LES AGENCES SAMIM.</p><buttoe salle de jeux en sous pente très lumineuse grâce à la présence de vastes vélux..
More information: ['Orientation Sud', 'Cheminée', 'Surface de 201,7m²', 'Bâtiment de 3 étages', '6 Pièces', '3 Chambres', 'Année de construction 1874', "2 Salles d'eau", '2 Toilettes', 'Chauffage gaz radiateur', 'Cuisine américaine équipée', 'Une entrée', 'Salle de séjour : 56m²', '1 Box']
Energy: ['131', '16']
Page Number: 60
-----------------------------------------------------
Category: Maison / Villa
City: Saint-Hippolyte-du-Fort
Criterion: ['6 pièces', '205 m²', '7 pièces']
Price: 295 000 €
Monthly: 1 225 €
</p><p> AGENCE SAMIM Christian CADEL / LES AGENCES SAMIM.</p><butto. BELLE RÉNOVATION. PROCHE BOUTIQUES VILLAGE - MARCHES HEBDOMADAIRESeuve avec garantie décennale

Category: Appartement
City: Quartier <!-- -->Les Termes<!-- -->, <!-- -->Mandelieu-la-Napoule
Criterion: ['2 pièces', '51.33 m²', '3 pièces']
Price: 199 000 €
Monthly: 829 €
Description: style="max-height: 66px;"><p>Superbe position pour ce beau et spacieux 2 pièces ! Venez découvrir ce nouveau bien entièrement rénové, situé dans une petite copropriété. Il offre un espace de vie de plus de 30 m² comprenant le séjour, salle à manger et sa cuisine ouverte aménagée au goût du jour, WC indépendant, chambre confortable et salle de douche. L'appartement est au calme et offre une jolie vue Mer. Agréable terrasse et jardin privatif. Stationnement facile, Faibles charges. Rare !. Ce bien vous est proposé en exclusivité par votre agence immobilière Orpi. Visite Virtuelle disponible en agence. 199000 euros Honoraires à la charge du vendeur.</p><butto
More information: ['1 Terrasse', 'Vue', 'Surface de 55,88m²', '2 Pièces', '1 Chambre', '1 Salle de bain', '1 Toilette', 'Toilette séparée', 'Chauffa

In [72]:
driver.quit()
driver2.quit()

In [73]:
full_data = pd.read_csv("./full_data.csv")
print(full_data.shape)

(897, 12)


In [74]:
# Delete duplicates.
full_data.drop_duplicates(keep='first', inplace=True)
full_data.to_csv("./full_data2.csv", index=False)
print(full_data.shape)

(707, 12)
